# Importamos las librerias necesarias# Torre de Hanoi - Algoritmo A*
# Introducción a la Inteligencia Artificial

## Problema de la Torre de Hanoi con 5 discos

### Descripción del problema
La Torre de Hanoi es un problema clásico que consiste en mover una torre de discos de diferentes tamaños de una varilla a otra, siguiendo estas reglas:
1. Solo se puede mover un disco a la vez
2. Un disco solo puede colocarse sobre otro disco más grande
3. Objetivo: mover todos los discos de la varilla izquierda (A) a la derecha (C)

### Estado inicial: Todos los discos en la varilla A
### Estado objetivo: Todos los discos en la varilla C

# Importamos las librerias necesarias

In [ ]:
import heapq
import time
import json
from typing import List, Tuple, Optional, Dict, Any
from datetime import datetime
import tracemalloc

# Definimos las clases necesarias

In [ ]:
class EstadoHanoi:
    """
    Representa un estado del problema de la Torre de Hanoi.
    Cada estado contiene las tres varillas con sus respectivos discos.
    """

    def __init__(self, varillas: List[List[int]]):
        self.varillas = [list(varilla) for varilla in varillas]  # Copia profunda

    def __eq__(self, other):
        return self.varillas == other.varillas

    def __hash__(self):
        return hash(tuple(tuple(varilla) for varilla in self.varillas))

    def __str__(self):
        return f"A:{self.varillas[0]} B:{self.varillas[1]} C:{self.varillas[2]}"

    def es_valido(self) -> bool:
        """Verifica si el estado actual es válido según las reglas de Hanoi"""
        for varilla in self.varillas:
            for i in range(len(varilla) - 1):
                if varilla[i] > varilla[i + 1]:  # Disco más grande sobre uno más pequeño
                    return False
        return True

    def obtener_movimientos_posibles(self) -> List['EstadoHanoi']:
        """Genera todos los estados posibles desde el estado actual"""
        movimientos = []

        for origen in range(3):
            if self.varillas[origen]:  # Si la varilla tiene discos
                disco = self.varillas[origen][-1]  # Disco superior

                for destino in range(3):
                    if origen != destino:
                        # Verificar si el movimiento es válido
                        if not self.varillas[destino] or self.varillas[destino][-1] > disco:
                            nuevo_estado = EstadoHanoi(self.varillas)
                            nuevo_estado.varillas[origen].pop()
                            nuevo_estado.varillas[destino].append(disco)
                            movimientos.append(nuevo_estado)

        return movimientos

In [ ]:
class NodoAEstrella:
    """
    Nodo para el algoritmo A*.
    Contiene el estado, costos g y h, y referencia al nodo padre.
    """

    def __init__(self, estado: EstadoHanoi, g: int = 0, h: int = 0, padre: Optional['NodoAEstrella'] = None):
        self.estado = estado
        self.g = g  # Costo desde el inicio
        self.h = h  # Heurística
        self.f = g + h  # Función de evaluación
        self.padre = padre

    def __lt__(self, other):
        return self.f < other.f

In [ ]:
class TorreHanoiAStar:
    """
    Implementación del algoritmo A* para resolver la Torre de Hanoi.
    """

    def __init__(self, num_discos: int = 5):
        self.num_discos = num_discos
        self.estado_inicial = EstadoHanoi([list(range(num_discos, 0, -1)), [], []])
        self.estado_objetivo = EstadoHanoi([[], [], list(range(num_discos, 0, -1))])
        self.nodos_explorados = 0
        self.nodos_expandidos = 0

    def heuristica_creativa(self, estado: EstadoHanoi) -> int:
        """
        Heurística creativa: Combinación de múltiples factores

        Fundamentación:
        1. Discos mal ubicados: Penaliza discos que no están en la varilla objetivo
        2. Discos bloqueados: Penaliza discos que tienen otros encima en varillas incorrectas
        3. Orden incorrecto: Penaliza cuando los discos no están en orden correcto
        4. Distancia de movimiento: Considera qué tan lejos están los discos de su destino

        Esta heurística es admisible porque nunca sobreestima el costo real,
        ya que cada factor representa movimientos mínimos necesarios.
        """
        h = 0

        # Factor 1: Discos que no están en la varilla objetivo (C)
        discos_mal_ubicados = 0
        for i in range(2):  # Varillas A y B
            discos_mal_ubicados += len(estado.varillas[i])

        # Factor 2: Discos bloqueados en varillas incorrectas
        discos_bloqueados = 0
        for i in range(2):  # Solo varillas A y B
            varilla = estado.varillas[i]
            for j, disco in enumerate(varilla):
                # Si hay discos encima, está bloqueado
                if j < len(varilla) - 1:
                    discos_bloqueados += 1

        # Factor 3: Verificar orden en varilla objetivo
        orden_incorrecto = 0
        varilla_c = estado.varillas[2]
        for i in range(len(varilla_c)):
            if varilla_c[i] != self.num_discos - i:
                orden_incorrecto += 1

        # Factor 4: Peso por tamaño de disco (discos más grandes son más costosos de mover)
        peso_discos = 0
        for i in range(2):
            for disco in estado.varillas[i]:
                peso_discos += disco * 0.1  # Pequeño peso adicional

        # Combinación de factores
        h = discos_mal_ubicados * 2 + discos_bloqueados + orden_incorrecto + peso_discos

        return int(h)

    def heuristica_simple(self, estado: EstadoHanoi) -> int:
        """
        Heurística simple propuesta: -1 por cada disco en posición correcta
        (Convertida a positiva para A*)
        """
        discos_correctos = len(estado.varillas[2])  # Discos en varilla C
        return self.num_discos - discos_correctos

    def a_estrella(self, usar_heuristica_creativa: bool = True) -> Tuple[List[EstadoHanoi], dict]:
        """
        Implementación del algoritmo A*.

        Returns:
            - Lista de estados que forman la solución
            - Diccionario con estadísticas del algoritmo
        """
        print(f"Iniciando A* con heurística {'creativa' if usar_heuristica_creativa else 'simple'}...")
        print(f"Estado inicial: {self.estado_inicial}")
        print(f"Estado objetivo: {self.estado_objetivo}")
        print("-" * 60)

        inicio_tiempo = time.time()

        # Inicialización
        heuristica = self.heuristica_creativa if usar_heuristica_creativa else self.heuristica_simple
        nodo_inicial = NodoAEstrella(self.estado_inicial, 0, heuristica(self.estado_inicial))

        lista_abierta = [nodo_inicial]
        lista_cerrada = set()
        self.nodos_explorados = 0
        self.nodos_expandidos = 0

        while lista_abierta:
            # Seleccionar nodo con menor f
            nodo_actual = heapq.heappop(lista_abierta)
            self.nodos_explorados += 1

            # Verificar si alcanzamos el objetivo
            if nodo_actual.estado == self.estado_objetivo:
                tiempo_total = time.time() - inicio_tiempo

                # Reconstruir camino
                camino = []
                nodo = nodo_actual
                while nodo:
                    camino.append(nodo.estado)
                    nodo = nodo.padre
                camino.reverse()

                estadisticas = {
                    'movimientos': len(camino) - 1,
                    'nodos_explorados': self.nodos_explorados,
                    'nodos_expandidos': self.nodos_expandidos,
                    'tiempo_ejecucion': tiempo_total,
                    'costo_solucion': nodo_actual.g,
                    'heuristica_usada': 'creativa' if usar_heuristica_creativa else 'simple'
                }

                return camino, estadisticas

            # Agregar a lista cerrada
            lista_cerrada.add(nodo_actual.estado)

            # Expandir nodos sucesores
            self.nodos_expandidos += 1
            for estado_sucesor in nodo_actual.estado.obtener_movimientos_posibles():
                if estado_sucesor not in lista_cerrada:
                    g_nuevo = nodo_actual.g + 1
                    h_nuevo = heuristica(estado_sucesor)
                    nodo_sucesor = NodoAEstrella(estado_sucesor, g_nuevo, h_nuevo, nodo_actual)

                    # Verificar si ya está en lista abierta con mejor costo
                    en_abierta = False
                    for i, nodo in enumerate(lista_abierta):
                        if nodo.estado == estado_sucesor:
                            if nodo_sucesor.f < nodo.f:
                                lista_abierta[i] = nodo_sucesor
                                heapq.heapify(lista_abierta)
                            en_abierta = True
                            break

                    if not en_abierta:
                        heapq.heappush(lista_abierta, nodo_sucesor)

        return None, {'error': 'No se encontró solución'}

# Definimos los métodos necesarios

In [ ]:
def mostrar_solucion(camino: List[EstadoHanoi]):
    """Muestra la solución paso a paso"""
    print("=== SOLUCIÓN ENCONTRADA ===\n")

    for i, estado in enumerate(camino):
        print(f"Paso {i}:")
        print(f"  A: {estado.varillas[0]}")
        print(f"  B: {estado.varillas[1]}")
        print(f"  C: {estado.varillas[2]}")

        if i < len(camino) - 1:
            # Determinar qué movimiento se hizo
            estado_actual = camino[i]
            estado_siguiente = camino[i + 1]

            movimiento = encontrar_movimiento(estado_actual, estado_siguiente)
            print(f"  → {movimiento}")

        print()

In [ ]:
def encontrar_movimiento(estado_antes: EstadoHanoi, estado_despues: EstadoHanoi) -> str:
    """Determina qué movimiento se realizó entre dos estados (para visualización)"""
    nombres = ['A', 'B', 'C']

    for origen in range(3):
        for destino in range(3):
            if len(estado_antes.varillas[origen]) > len(estado_despues.varillas[origen]) and \
               len(estado_antes.varillas[destino]) < len(estado_despues.varillas[destino]):
                disco = estado_despues.varillas[destino][-1]
                return f"Mover disco {disco} de {nombres[origen]} a {nombres[destino]}"
    return "Movimiento no identificado"

In [ ]:
def determinar_movimiento(estado_antes: EstadoHanoi, estado_despues: EstadoHanoi) -> Dict[str, Any]:
    """
    Determina el movimiento entre dos estados.

    Returns:
        Dict con formato: {
            "type": "movement",
            "disk": int,
            "peg_start": int,
            "peg_end": int
        }
    """

    for origen in range(3):
        for destino in range(3):
            if origen != destino:
                # Verificar si se movió un disco de origen a destino
                if (len(estado_antes.varillas[origen]) > len(estado_despues.varillas[origen]) and
                    len(estado_antes.varillas[destino]) < len(estado_despues.varillas[destino])):

                    # El disco movido es el que ahora está en el tope del destino
                    disco_movido = estado_despues.varillas[destino][-1]

                    return {
                        "type": "movement",
                        "disk": disco_movido,
                        "peg_start": origen + 1,  # Convertir a 1-indexed
                        "peg_end": destino + 1    # Convertir a 1-indexed
                    }

    return None

In [ ]:
def exportar_solucion(camino: List[EstadoHanoi], estadisticas: Dict[str, Any]) -> None:
    """
    Exporta la solución encontrada en el formato específico requerido por
    la herramienta de visualización.

    Genera dos archivos:
    1. initial_state.json: Estado inicial del problema
    2. sequence.json: Secuencia de movimientos

    Formato:

    initial_state.json:
    {
        "peg_1": [5, 4, 3, 2, 1],
        "peg_2": [],
        "peg_3": []
    }

    sequence.json:
    [
        {
            "type": "movement",
            "disk": 1,
            "peg_start": 1,
            "peg_end": 3
        },
        ...
    ]
    """

    # 1. CREAR ARCHIVO initial_state.json
    estado_inicial = camino[0]
    initial_state = {
        "peg_1": list(estado_inicial.varillas[0]),  # Varilla izquierda (A)
        "peg_2": list(estado_inicial.varillas[1]),  # Varilla del medio (B)
        "peg_3": list(estado_inicial.varillas[2])   # Varilla derecha (C)
    }

    try:
        with open('initial_state.json', 'w', encoding='utf-8') as archivo:
            json.dump(initial_state, archivo, indent=2, ensure_ascii=False)
        print("✓ Archivo 'initial_state.json' creado exitosamente")
    except Exception as e:
        print(f"✗ Error al crear initial_state.json: {e}")
        return

    # 2. CREAR ARCHIVO sequence.json
    sequence = []

    # Generar secuencia de movimientos
    for i in range(1, len(camino)):
        estado_anterior = camino[i-1]
        estado_actual = camino[i]

        # Encontrar qué disco se movió y entre qué varillas
        movimiento = determinar_movimiento(estado_anterior, estado_actual)

        if movimiento:
            sequence.append(movimiento)
        else:
            print(f"⚠️  Advertencia: No se pudo determinar movimiento en paso {i}")

    try:
        with open('sequence.json', 'w', encoding='utf-8') as archivo:
            json.dump(sequence, archivo, indent=2, ensure_ascii=False)
        print("✓ Archivo 'sequence.json' creado exitosamente")
    except Exception as e:
        print(f"✗ Error al crear sequence.json: {e}")
        return

    print(f"   Algoritmo: A* con heurística {estadisticas.get('heuristica_usada', 'creativa')}")
    print(f"   Tiempo de ejecución: {estadisticas.get('tiempo_ejecucion', 0):.4f} segundos")
    print(f"   Nodos explorados: {estadisticas.get('nodos_explorados', 0)}")

In [ ]:
def validar_archivos() -> bool:
    """
    Valida que los archivos generados tengan el formato correcto.
    """

    # 1. Validar initial_state.json
    print("1. Validando initial_state.json...")
    try:
        with open('initial_state.json', 'r', encoding='utf-8') as archivo:
            initial_state = json.load(archivo)

        # Verificar estructura
        required_keys = ['peg_1', 'peg_2', 'peg_3']
        if not all(key in initial_state for key in required_keys):
            print("   ✗ Estructura incorrecta: faltan claves requeridas")
            return False

        # Verificar que todos los valores sean listas
        for key in required_keys:
            if not isinstance(initial_state[key], list):
                print(f"   ✗ {key} debe ser una lista")
                return False

        # Validar discos
        todos_discos = []
        for peg in ['peg_1', 'peg_2', 'peg_3']:
            todos_discos.extend(initial_state[peg])

        # Verificar discos únicos
        if len(todos_discos) != len(set(todos_discos)):
            print("   ✗ Hay discos duplicados")
            return False

        # Verificar secuencia completa
        if todos_discos and (set(todos_discos) != set(range(1, max(todos_discos) + 1))):
            print("   ✗ Secuencia de discos incompleta")
            return False

        print("   ✓ initial_state.json válido")

    except FileNotFoundError:
        print("   ✗ Archivo initial_state.json no encontrado")
        return False
    except json.JSONDecodeError:
        print("   ✗ Error al decodificar initial_state.json")
        return False

    # 2. Validar sequence.json
    print("2. Validando sequence.json...")
    try:
        with open('sequence.json', 'r', encoding='utf-8') as archivo:
            sequence = json.load(archivo)

        # Verificar que sea una lista
        if not isinstance(sequence, list):
            print("   ✗ sequence.json debe ser una lista")
            return False

        # Validar cada movimiento
        for i, move in enumerate(sequence):
            if not isinstance(move, dict):
                print(f"   ✗ Movimiento {i} debe ser un objeto")
                return False

            required_move_keys = ['type', 'disk', 'peg_start', 'peg_end']
            if not all(key in move for key in required_move_keys):
                print(f"   ✗ Movimiento {i} incompleto")
                return False

            if move['type'] != 'movement':
                print(f"   ✗ Movimiento {i} debe tener type='movement'")
                return False

            # Verificar que los números de varilla sean válidos
            if not (1 <= move['peg_start'] <= 3 and 1 <= move['peg_end'] <= 3):
                print(f"   ✗ Movimiento {i} tiene números de varilla inválidos")
                return False

        print(f"   ✓ sequence.json válido ({len(sequence)} movimientos)")

    except FileNotFoundError:
        print("   ✗ Archivo sequence.json no encontrado")
        return False
    except json.JSONDecodeError:
        print("   ✗ Error al decodificar sequence.json")
        return False

    print("\n🎉 Ambos archivos son válidos!")
    return True

In [ ]:
def generar_archivos_completo(camino: List[EstadoHanoi], estadisticas: Dict[str, Any]):
    """
    Función principal para generar archivos para visualización de la solución.
    """

    # Generar archivos
    exportar_solucion(camino, estadisticas)

    # Validar archivos
    if validar_archivos():
        return True
    else:
        print("\n❌ Error en la generación de archivos")
        return False

In [ ]:
def comparar_heuristicas():
    n = 5 #número de discos de la torre
    """Compara el rendimiento de ambas heurísticas"""
    torre = TorreHanoiAStar(n)

    # Probar heurística simple
    print("\n1. HEURÍSTICA SIMPLE (discos fuera de lugar)")
    camino_simple, stats_simple = torre.a_estrella(usar_heuristica_creativa=False)

    if camino_simple:
        print(f"✓ Solución encontrada en {stats_simple['movimientos']} movimientos")
        print(f"  - Nodos explorados: {stats_simple['nodos_explorados']}")
        print(f"  - Nodos expandidos: {stats_simple['nodos_expandidos']}")
        print(f"  - Tiempo: {stats_simple['tiempo_ejecucion']:.4f} segundos")

    # Probar heurística creativa
    print("\n2. HEURÍSTICA CREATIVA (multifactor)")
    torre = TorreHanoiAStar(n)  # Nueva instancia
    camino_creativo, stats_creativo = torre.a_estrella(usar_heuristica_creativa=True)

    if camino_creativo:
        print(f"✓ Solución encontrada en {stats_creativo['movimientos']} movimientos")
        print(f"  - Nodos explorados: {stats_creativo['nodos_explorados']}")
        print(f"  - Nodos expandidos: {stats_creativo['nodos_expandidos']}")
        print(f"  - Tiempo: {stats_creativo['tiempo_ejecucion']:.4f} segundos")

    # Análisis comparativo
    print("\n" + "="*50)
    print("ANÁLISIS COMPARATIVO")
    print("="*50)

    if camino_simple and camino_creativo:
        if camino_simple == camino_creativo: print(f"Ambas encontraron solución óptima: {stats_simple['movimientos']} movimientos")
        print(f"Eficiencia en nodos explorados:")
        print(f"  - Simple: {stats_simple['nodos_explorados']} nodos")
        print(f"  - Creativa: {stats_creativo['nodos_explorados']} nodos")
        print(f"  - Mejora: {((stats_simple['nodos_explorados'] - stats_creativo['nodos_explorados']) / stats_simple['nodos_explorados'] * 100):.1f}%")

    return camino_creativo, stats_creativo

# ==========================================
# EJECUCIÓN PRINCIPAL
# ==========================================

In [ ]:
##%%timeit
if __name__ == "__main__":
    print("TORRE DE HANOI - ALGORITMO A*")
    print("Problema: 5 discos, de varilla A a varilla C")
    print("=" * 80)

    debug = False
    # Ejecutar comparación de heurísticas

    solucion_final, estadisticas_finales = comparar_heuristicas()

    # Generar archivos para visualización FIUBA
    if solucion_final:
        exito = generar_archivos_completo(solucion_final, estadisticas_finales)

    # Mostrar la solución detallada
    if solucion_final and debug:
        print("\n" + "=" * 80)
        mostrar_solucion(solucion_final)

## =============================================================================
## FUNDAMENTOS DE LA HEURÍSTICA ELEGIDA
## =============================================================================

        HEURÍSTICA CREATIVA SELECCIONADA:

        La heurística implementada combina cuatro factores clave:

        1. DISCOS MAL UBICADOS (peso: 2x):
           - Cuenta discos que no están en la varilla objetivo (C)
           - Fundamental porque cada disco debe moverse al menos una vez

        2. DISCOS BLOQUEADOS (peso: 1x):
           - Penaliza discos que tienen otros encima en varillas incorrectas
           - Importante porque requieren movimientos adicionales para liberarse

        3. ORDEN INCORRECTO (peso: 1x):
           - Verifica si los discos en C están en el orden correcto
           - Evita soluciones subóptimas con discos desordenados

        4. PESO POR TAMAÑO (peso: 0.1x):
           - Considera que mover discos grandes es conceptualmente más costoso
           - Proporciona desempate para situaciones similares

        PROPIEDADES:
        - ADMISIBLE: Nunca sobreestima el costo real
        - CONSISTENTE: Cumple la desigualdad triangular
        - INFORMATIVA: Guía eficientemente hacia la solución

        Esta heurística mejora significativamente el rendimiento al reducir
        el espacio de búsqueda sin sacrificar la optimalidad de la solución.